In [4]:
import torch
import torch.nn as nn

import torchvision.transforms as transforms
import torch.utils.data as data_utils
from torch.utils.data import TensorDataset, DataLoader
from torchvision.datasets.mnist import MNIST
from torchvision import datasets

from model import Generator, Discriminator
from train import train_DCGAN

device = torch.device("cuda" if torch.cuda.is_available()  else "cpu")

In [2]:
import numpy as np
with open("numpy_data.npy","rb") as f:
    pixels = np.load(f)

In [3]:
pixels = np.resize(pixels,(1201,1,128,128))

In [4]:
pixels.shape

(1201, 1, 128, 128)

In [13]:
trans = transforms.Compose([
            transforms.Scale(128),
            transforms.ToTensor(),
            transforms.Normalize((0.5, ), (0.5, )),
        ])
path = r'C:\Users\Monalisha\Desktop\generative modelling\project'
dataset = datasets.ImageFolder(path, transform=trans)

Hyperparameters are listed here. You maybe need to play a little bit with.

In [14]:
dataset

Dataset ImageFolder
    Number of datapoints: 1889
    Root location: C:\Users\Monalisha\Desktop\generative modelling\project
    StandardTransform
Transform: Compose(
               Scale(size=128, interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

In [16]:
g_lr = 0.0002
d_lr = 0.0002
batch_size = 128
num_epochs = 10

In [17]:

#train_set = MNIST(root='.', train=True, transform=trans, download=True)
train_loader = data_utils.DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [20]:
images, labels = next(iter(train_loader))

In [22]:
images[0]

tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]],

        [[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]],

        [[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]]])

In [14]:
input1 = torch.randn(32, 1, 128, 128)
net = nn.Sequential(
            # input size batch x 1 x 128 x 128
            nn.Conv2d(1, 256, 4, 2, 1, bias=False) ,
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            # input size batch x 256 
            nn.Conv2d(256, 512, 4, 2, 1, bias=False) ,
            nn.BatchNorm2d(512),
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            nn.Flatten(),
            # input size batch x 8192 
            nn.Linear(524288,1),
            nn.Sigmoid()
)

output = net(input1)
output.size()

torch.Size([32, 1])

In [ ]:


G = Generator().to(device)
D = Discriminator().to(device)

optim_G = torch.optim.Adam(G.parameters(), lr=g_lr, betas=(0.5, 0.999))
optim_D = torch.optim.Adam(D.parameters(), lr=d_lr, betas=(0.5, 0.999))

loss_f = nn.BCELoss()

train_DCGAN(G, D, optim_G, optim_D, loss_f, train_loader, num_epochs, device)

C:\Users\Monalisha\anaconda3\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
